# Introduction



Today, we are excited to announce a Kubernetes Operator to increase airflow's viability as a job orchestration engine using the power of the Kubernetes cloud deployment framework. 

Since its inception, airflow's power as a job orchestrator has been its flexibility. Airflow offers a wide range of native operators (for services such as spark, hbase, etc.) while also offering easy extensibility through its plugin framework. However, one limitation of the Apache Airflow project has been that it has always relied on previously running systems. 

Over the next few months, we hope to change this model using containerization. With a kubernetes operator, airflow is no longer limited by the size of the worker nodes, or the libararies loaded to the airflow instances.

## What is kubernetes? (Is this section necessary?)

* Kubernetes is a container orchestration system
* users can launch docker containers on custom sized instnaces
* 

# The Kubernetes Operator: The "Whatever-your-heart-desires" Operator

## Example 1: Running a basic container

For this first example, let's create a basic docker container that runs simple python commands. This container should be able to pass and fail at will, and record logs that show up in the airflow terminal.

In [1]:
import sys


def fail():
    return 10 / 0


def succeed():
    return 10 / 1


if __name__ == "__main__":
    if len(sys.argv) != 2:
        print("Usage: wordcount <file>", file=sys.stderr)
        exit(-1)
    arg = sys.argv[1]
    if arg == 'pass':
        succeed()
    elif arg == 'fail':
        fail()
    else:
        print('invalid command: {}'.format(arg))
        exit(-1)

Paragraph discussing creating a docker container for running this scipy code using an entry point.

In [ ]:
FROM ubuntu:16.04

# install python dependencies
RUN apt-get update -y && apt-get install -y \
        wget \
        python-dev \
        python-pip \
        libczmq-dev \
        libcurlpp-dev \
        curl \
        libssl-dev \
        git \
        inetutils-telnet \
        bind9utils

RUN pip install -U setuptools && \
    pip install -U pip


COPY airflow-example.py /airflow-example.py
COPY entrypoint.sh /entrypoint.sh
RUN chmod +x /entrypoint.sh

ENTRYPOINT ["/entrypoint.sh"]

Discussion of entrypoint files 

In [ ]:
#!/usr/bin/env bash

python /airflow-example.py

In [ ]:
# Airflow DAG File that creates two kubernetes operators, one that passes and one that fails

passing = KubernetesPodOperator(namespace='default',
                          image="airflow-test",
                          arguments=["succeed"],
                          labels={"foo": "bar"},
                          name="passing-test",
                          task_id="task1"
                          )

failing = KubernetesPodOperator(namespace='default',
                          image="airflow-test",
                          arguments=["succeed"],
                          labels={"foo": "bar"},
                          name="fail",
                          task_id="task1"
                          )

failing.set_upstream(passing)

### This will eventually be a series of images about the running DAGs

<img src="files/image.png">

Link to github file

## Example 2: Running a model using scipy

## Example 3: Running a kubeflow pipeline


Paragraph about kubeflow and how awesome it will be

* Developed by Google and Bloomberg to make kubernetes data development easier

## How it Works

# Closing Statements

Final statements about all the possibilities this opens up

* Airflow Kubernetes Executor
* Custom Deployments via python API